In [1]:
import gurobipy as gp
from gurobipy import GRB
from itertools import product

### Problem Data

In [12]:
supply = [400,610,340,350]
demand = [550,250,400,500]
num_supply = len(supply)
num_demand = len(demand)
cartesian_supply_demand = list(product(range(num_supply),range(num_demand)))
cost = [22,17,30,18,15,35,20,25,28,21,16,14,40,65,25,50]
acr_cost = {cartesian_supply_demand[i]:cost[i] for i in range(16)}

{(0, 0): 22, (0, 1): 17, (0, 2): 30, (0, 3): 18, (1, 0): 15, (1, 1): 35, (1, 2): 20, (1, 3): 25, (2, 0): 28, (2, 1): 21, (2, 2): 16, (2, 3): 14, (3, 0): 40, (3, 1): 65, (3, 2): 25, (3, 3): 50}


### Decision Variable

In [13]:
model = gp.Model('Modified network')

Restricted license - for non-production use only - expires 2025-11-24


In [14]:
num_arc_sent = model.addVars(acr_cost, vtype=GRB.CONTINUOUS, lb=0.00, name='num_arc_sent')

### Objective Function

In [16]:
model.setObjective(gp.quicksum(num_arc_sent[(i,j)]*acr_cost[(i,j)] for i in range(num_supply) for j in range(num_demand)))

### Constraints

In [21]:
# supply
supply_constraint = model.addConstrs((gp.quicksum(num_arc_sent[(i,j)] for j in range(num_demand)) <= supply[i] for i in range(num_supply)), name='Supply')
# demand
demand_constraint = model.addConstrs((gp.quicksum(num_arc_sent[(i,j)] for i in range(num_supply)) >= demand[j] for j in range(num_demand)), name='Demand')

In [22]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 24 rows, 16 columns and 96 nonzeros
Model fingerprint: 0x3fca1190
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve removed 16 rows and 0 columns
Presolve time: 0.01s
Presolved: 8 rows, 16 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.125000e+02   0.000000e+00      0s
       9    2.9960000e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.996000000e+04


In [23]:
model.printAttr('X')


    Variable            X 
-------------------------
num_arc_sent[0,1]          250 
num_arc_sent[0,3]          150 
num_arc_sent[1,0]          550 
num_arc_sent[1,2]           50 
num_arc_sent[1,3]           10 
num_arc_sent[2,3]          340 
num_arc_sent[3,2]          350 
